# Analyze Model Behvaiour of Classification

In [1]:
import numpy as np
import h5py
import pickle
from tqdm import tqdm
import matplotlib.pyplot as plt
from skimage.transform import resize

In [13]:
latents_values = dataset_zip['latents_values'] # 737280, 6
# 'color', 'shape', 'scale', 'orientation', 'posX', 'posY'